In [ ]:
import turtle
import random

In [ ]:
# Set up the screen
screen = turtle.Screen()
screen.title("Space Invaders")
screen.bgcolor("black")
screen.setup(width=800, height=600)
screen.tra

In [ ]:
class Player:
    def __init__(self):
        self.player = turtle.Turtle()
        self.player.speed(0)
        self.player.shape("triangle")
        self.player.color("white")
        self.player.penup()
        self.player.goto(0, -250)
        self.player.setheading(90)
        self.speed = 15

    def move_left(self):
        x = self.player.xcor()
        if x > -380:
            x -= self.speed
            self.player.setx(x)

    def move_right(self):
        x = self.player.xcor()
        if x < 380:
            x += self.speed
            self.player.setx(x)

In [ ]:
class Bullet:
    def __init__(self, x, y):
        self.bullet = turtle.Turtle()
        self.bullet.speed(0)
        self.bullet.shape("circle")
        self.bullet.color("yellow")
        self.bullet.penup()
        self.bullet.goto(x, y)
        self.bullet.setheading(90)
        self.speed = 20
        self.state = "ready"

    def move(self):
        if self.state == "fire":
            y = self.bullet.ycor()
            y += self.speed
            self.bullet.sety(y)

            if y > 280:
                self.state = "ready"
                self.bullet.hideturtle()

    def fire(self, x, y):
        if self.state == "ready":
            self.state = "fire"
            self.bullet.goto(x, y + 10)
            self.bullet

In [ ]:
class Invader:
    def __init__(self, x, y):
        self.invader = turtle.Turtle()
        self.invader.speed(0)
        self.invader.shape("square")
        self.invader.color("red")
        self.invader.penup()
        self.invader.goto(x, y)
        self.speed = 2

    def move(self):
        x = self.invader.xcor()
        x += self.speed
        self.invader.setx(x)

    def move_down(self):
        y = self.invader.ycor()
        y -= 40
        self.invader.sety(y)
        self.speed *= -1

In [ ]:
class Barrier:
    def __init__(self, x, y):
        self.barrier = turtle.Turtle()
        self.barrier.speed(0)
        self.barrier.shape("square")
        self.barrier.color("green")
        self.barrier.penup()
        self.barrier.goto(x, y)
        self.hits = 0
        self.max_hits = 3

    def hit(self):
        self.hits += 1
        if self.hits == 1:
            self.barrier.color("yellow")
        elif self.hits == 2:
            self.barrier.color("orange")
        elif self.hits >= self.max_hits:
            self.barrier.hideturtle()

In [ ]:
def is_collision(obj1, obj2):
    distance = ((obj1.xcor() - obj2.xcor())**2 + (obj1.ycor() - obj2.ycor())**2)**0.5


In [ ]:
# Create game objects
player = Player()
bullets = []
invaders = []
barriers = []

# Create invaders in formation
for row in range(5):
    for col in range(11):
        invader = Invader(-250 + col * 50, 200 - row * 50)
        invaders.append(invader)

# Create barriers
for i in range(4):
    for j in range(3):
        barrier = Barrier(-300 + i * 200 + j * 20, -150)
        barriers.append(barrier)

# Game variables
score = 0
game_over = False
invader_direction = 1
move_counter = 0

# Score display
score_pen = turtle.Turtle()
score_pen.speed(0)
score_pen.color("white")
score_pen.penup()
score_pen.goto(-390, 260)
score_pen.hideturtle()

def update_score():
    score_pen.clear()
    score_pen.write(f"Score: {score}", align="left", font=("Arial", 16, "normal"))

In [ ]:
# Keyboard bindings
def move_left():
    player.move_left()

def move_right():
    player.move_right()

def fire_bullet():
    bullet = Bullet(player.player.xcor(), player.player.ycor())
    bullets.append(bullet)
    bullet.fire(player.player.xcor(), player.player.ycor())

screen.listen()
screen.onkey(move_left, "Left")
screen.onkey(move_right, "Right")
screen.onkey(fire_bullet, "space")

In [ ]:
# Main game loop
try:
    while not game_over and invaders:
        screen.update()
        time.sleep(0.02)

        move_counter += 1

        # Move invaders
        if move_counter % 30 == 0:  # Move every 30 frames (about 0.6 seconds)
            edge_hit = False
            for invader in invaders:
                invader.move()
                if invader.invader.xcor() > 380 or invader.invader.xcor() < -380:
                    edge_hit = True

            if edge_hit:
                for invader in invaders:
                    invader.move_down()

        # Move bullets
        for bullet in bullets[:]:  # Use slice to avoid modification during iteration
            bullet.move()
            if bullet.state == "ready":
                bullets.remove(bullet)

        # Check collisions between bullets and invaders
        for bullet in bullets[:]:
            for invader in invaders[:]:
                if bullet.state == "fire" and is_collision(bullet.bullet, invader.invader):
                    # Remove bullet and invader
                    bullet.bullet.hideturtle()
                    bullets.remove(bullet)
                    invader.invader.hideturtle()
                    invaders.remove(invader)
                    score += 10
                    update_score()
                    break

        # Check collisions between bullets and barriers
        for bullet in bullets[:]:
            for barrier in barriers[:]:
